In [133]:
import numpy as np
from scipy.linalg import expm

In [134]:
SigX = np.array([[0, 1], [1, 0]])
SigY = np.array([[0, -1j], [1j, 0]])
SigZ = np.array([[1, 0], [0, -1]])
SigI = np.array([[1, 0], [0, 1]])

def model_Gxpi2(overrot, delta):
    return expm(-(1j/2)*((np.pi/2 + overrot)*SigX + delta*SigZ))

def model_Gzpi2(epsilon):
    return expm(-(1j/2)*(np.pi/2 + epsilon)*SigZ)

def sequence_to_unitary(sequence, x_params):
    x_overrot = x_params[0]
    x_delta = x_params[1]
    z_epsilon = x_params[2]
    U = SigI
    for gate in sequence:
        if gate == 'X':
            U = model_Gxpi2(x_overrot, x_delta) @ U
        elif gate == 'Z':
            U = model_Gzpi2(z_epsilon) @ U
    return U

In [145]:
def calc_normalized_eigenmatrix(matrix):
    # Calculate the eigenvalues and eigenvectors of the matrix
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    # Normalize the eigenvectors
    eigen_norm = np.linalg.norm(eigenvectors, axis=1)
    # Return the normalized eigenvectors
    return eigenvectors/eigen_norm

def calc_eigen_jacobian_at_target(sequence, num_params, hilbert_dim=2):
    """
    Calculate the Jacobian of the eigenvalues of the target unitary matrix with respect to the parameters

    Returns a matrix of shape (hilbert_dim, num_params) where the (i, j) element is the derivative of the ith eigenvalue
    with respect to the jth parameter.
    """
    # Define the target unitary matrix
    target_params = np.zeros(num_params)
    target_unitary = sequence_to_unitary(sequence, target_params)
    assert target_unitary.shape[0] == target_unitary.shape[1]
    assert target_unitary.shape[0] == hilbert_dim

    # Calculate the normalized eigenvectors of the target unitary matrix
    normalized_eigenvectors = calc_normalized_eigenmatrix(target_unitary)

    # Calculate the derivative of the eigenvalues with respect to the parameters
    jacobian = np.zeros((hilbert_dim, num_params), dtype=np.complex128)

    for i in range(hilbert_dim):
        # Calculate the derivs w.r.t. the ith parameter
        for j in range(num_params):
            # Define the parameter perturbations
            perturbations = np.zeros(num_params)
            perturbations[j] = 1e-8

            # Calculate the perturbed unitary matrices
            perturbed_unitary_plus = sequence_to_unitary(sequence, target_params + perturbations)
            perturbed_unitary_minus = sequence_to_unitary(sequence, target_params - perturbations)

            # Calculate the perturbed eigenvectors
            eval_plus = normalized_eigenvectors[i].conj().T @ perturbed_unitary_plus @ normalized_eigenvectors[i]
            eval_minus = normalized_eigenvectors[i].conj().T @ perturbed_unitary_minus @ normalized_eigenvectors[i]

            # Calculate the derivative
            jacobian[i, j] = (eval_plus - eval_minus)/(2*perturbations[j])
    return jacobian

In [136]:
Ztarget = model_Gzpi2(0.)
Ztarget @ Ztarget.conj().T

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [137]:
eigen_mat_X = calc_normalized_eigenmatrix(model_Gxpi2(0, 0))
np.conj(eigen_mat_X).T@ eigen_mat_X

array([[1.00000000e+00+0.0000000e+00j, 1.11022302e-16-1.3738309e-16j],
       [1.11022302e-16+1.3738309e-16j, 1.00000000e+00+0.0000000e+00j]])

In [138]:
calc_eigen_jacobian_at_target(['X', 'X', 'Z', 'Z'], 3, 2)

array([[-1.65436120e-16-1.11022302e-08j, -6.36619772e-01+0.00000000e+00j,
        -2.06795153e-16-1.11022302e-08j],
       [-1.65436119e-16+1.11022302e-08j, -6.36619772e-01+0.00000000e+00j,
        -2.48154184e-16+0.00000000e+00j]])

In [139]:
calc_eigen_jacobian_at_target(['X', 'X', 'Z', 'Z'], 3, 2)

array([[-1.65436120e-16-1.11022302e-08j, -6.36619772e-01+0.00000000e+00j,
        -2.06795153e-16-1.11022302e-08j],
       [-1.65436119e-16+1.11022302e-08j, -6.36619772e-01+0.00000000e+00j,
        -2.48154184e-16+0.00000000e+00j]])

In [140]:
calc_eigen_jacobian_at_target(['Z', 'X', 'Z', 'Z', 'Z'], 3, 2)

array([[ 3.53553392e-01-3.53553387e-01j,  5.55111512e-09-1.11022302e-08j,
        -5.55111512e-09-2.22044605e-08j],
       [ 3.53553387e-01+3.53553398e-01j,  0.00000000e+00+0.00000000e+00j,
        -5.55111512e-09+1.11022302e-08j]])

In [141]:
calc_eigen_jacobian_at_target(['Z', 'Z', 'X', 'Z', 'Z', 'X'], 3, 2)

array([[-1.11022302e-08-2.68882139e-09j,  0.00000000e+00+9.54061940e-01j,
        -2.22044605e-08+1.49863699e+00j],
       [-1.11022302e-08+1.96261549e-09j,  0.00000000e+00-6.36619772e-01j,
        -1.66533454e-08-9.99999998e-01j]])

In [142]:
def score_eigen_jacobian_matrix(jacobian_matrix):
    svals = np.linalg.svd(jacobian_matrix, compute_uv=False)
    svals_clipped = np.clip(svals, 1e-8, None)
    return np.sum(np.log(svals_clipped))

In [143]:
sequences = [['X'], ['Z', 'X', 'Z', 'Z', 'Z'], ['Z', 'Z', 'X', 'Z', 'Z', 'X']]
num_params = 3
hilbert_dim = 2
total_jac = np.zeros((hilbert_dim, 0), dtype=np.complex128)
for sequence in sequences:
    jac = calc_eigen_jacobian_at_target(sequence, num_params, hilbert_dim)
    total_jac = np.concatenate((total_jac, jac), axis=1)
score_eigen_jacobian_matrix(total_jac)

0.4642511033955037

In [144]:
score_eigen_jacobian_matrix(calc_eigen_jacobian_at_target(['X', 'X', 'Z', 'Z'], 3, 2))

-17.962983574797143